In [1]:
import os
from pyspark.sql.functions import col, lit, when, from_json
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StringType,IntegerType, ArrayType
import pyspark
from ast import literal_eval
from functools import reduce
from pyspark.sql.functions import col, lit, when, from_json
from graphframes import *
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import networkx as nx

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell graphframes:graphframes-0.6'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SSKafka") \
    .getOrCreate()
    
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
dsraw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "172.23.149.197:9092") \
  .option("subscribe", "cardnode.public.ada_pots") \
  .option("startingOffsets", "earliest") \
  .load()

22/05/28 15:58:07 WARN Utils: Your hostname, cardano-druid resolves to a loopback address: 127.0.0.1; using 172.23.149.210 instead (on interface ens3)
22/05/28 15:58:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7692199b-93e0-47c0-a87c-c6e054b36523;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-log

In [4]:
ds = dsraw.selectExpr("CAST(value AS STRING)")

In [40]:
# Subscribe to 1 topic, with headers
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2") \
  .option("subscribe", "topic1") \
  .option("includeHeaders", "true") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "headers")

22/05/23 12:47:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3b45a15c-823c-4b95-8846-e54cad70c898. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/23 12:47:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/23 12:47:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-5bccbf21-d550-4f23-bfde-2088ac7d622c-1837894314-driver-0-11, groupId=spark-kafka-source-5bccbf21-d550-4f23-bfde-2088ac7d622c-1837894314-driver-0] Connection to node -1 (/172.23.149.197:9092) could not be established. Broker may not be available.
22/05/23 12:47:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-5bccbf21-d550-4f23-bfde-2088ac7d622c-1837894314-driver-

In [5]:
import ast

In [6]:
import json

s = []
with open('notebook/part-00000-69f29234-ab7d-4fb0-9756-e53ba6c0ebfc-c000.json') as f:
    for line in f:
        s.append(ast.literal_eval(line)["value"])

In [7]:
# INPUT

path0 = "/home/ubuntu/mock_data/NodePostgresTablesJSON"

## list of the years to extract
extracted_years=glob(path0+"/*.json")
years=[year.split('/')[-1] for year in extracted_years]

## Query: put together transaction in and transaction out

In [10]:
tx_in = spark.read.json("/home/ubuntu/tx_in_10k.json")
tx_in.createOrReplaceTempView("tx_in")

In [11]:
tx = spark.read.json("/home/ubuntu/tx_10k.json")
tx.createOrReplaceTempView("tx")

In [15]:
tx_out = spark.read.json("/home/ubuntu/tx_out_10k.json")
tx_out.createOrReplaceTempView("tx_out")

### Transaction outputs for specified transaction hash:

In [29]:
qr_out = "select tx_out.* from tx_out inner join tx on tx_out.tx_id = tx.id"

In [30]:
spark.sql(qr_out).createOrReplaceTempView("qr_out")

In [31]:
spark.sql(qr_out).show()

+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+
|_corrupt_record|             address|address_has_script|         address_raw|data_hash|     id|index|        payment_cred|stake_address_id|  tx_id|        value|
+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+
|           null|addr1qx5dhehk7edv...|             false|\x01a8dbe6f6f65ac...|     null|6844725|    0|\xa8dbe6f6f65acfc...|          139423|2851666|    865644802|
|           null|DdzFFzCqrhsdwfxt4...|             false|\x82d818584283581...|     null|6844726|    0|                null|            null|2851667|     10000000|
|           null|Ae2tdPwUPEZMhScXD...|             false|\x82d818582183581...|     null|6844727|    1|                null|            null|2851667| 636988708079|
|           null|addr1

In [34]:
qr3 = "select qr_out.id, qr_out.address as address_output, tx.hash from qr_out inner join tx on tx.id = qr_out.tx_id"

In [35]:
spark.sql(qr3).show()

+-------+--------------------+--------------------+
|     id|      address_output|                hash|
+-------+--------------------+--------------------+
|6844725|addr1qx5dhehk7edv...|\x26f0159f7172586...|
|6844726|DdzFFzCqrhsdwfxt4...|\x02f60c84eeb8831...|
|6844727|Ae2tdPwUPEZMhScXD...|\x02f60c84eeb8831...|
|6844728|addr1qyydz7karapt...|\x0d5501e66e7d2e1...|
|6844729|addr1q9gjp5cftwu7...|\xed0ca4ff835da89...|
|6844730|addr1q83l0h40rd56...|\x8b879dce5cc2db7...|
|6844731|Ae2tdPwUPEZ45YFwo...|\xf30e11bcca2d0f2...|
|6844732|DdzFFzCqrhsgvV4Hx...|\xf30e11bcca2d0f2...|
|6844733|DdzFFzCqrhsg6RRF2...|\xc4c4d4062c2699c...|
|6844734|DdzFFzCqrhszeaQtA...|\x4ff065b2fa59e8e...|
|6844735|Ae2tdPwUPEZ317wJQ...|\x4ff065b2fa59e8e...|
|6844736|addr1q8asgl3htazr...|\xe646a153a288239...|
|6844737|Ae2tdPwUPEZ5D7W1r...|\xe646a153a288239...|
|6844738|Ae2tdPwUPEZF7693F...|\xe646a153a288239...|
|6844739|addr1q9hfwzsq79y2...|\xe646a153a288239...|
|6844740|DdzFFzCqrhsduTydx...|\xe646a153a288239...|
|6844741|Ae2

In [55]:
spark.sql(qr3).createOrReplaceTempView("qr3")

In [56]:
qr_in = "select tx_out.*, tx.hash from tx_out \
    inner join tx_in on tx_out.tx_id = tx_in.tx_out_id \
    inner join tx on tx.id = tx_in.tx_in_id and tx_in.tx_out_index = tx_out.index"

In [57]:
spark.sql(qr_in).createOrReplaceTempView("qr_in")

In [58]:
spark.sql(qr_in).show()

+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+--------------------+
|_corrupt_record|             address|address_has_script|         address_raw|data_hash|     id|index|        payment_cred|stake_address_id|  tx_id|        value|                hash|
+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+--------------------+
|           null|Ae2tdPwUPEZMhScXD...|             false|\x82d818582183581...|     null|6844727|    1|                null|            null|2851667| 636988708079|\x32748b8f047aff1...|
|           null|addr1qyydz7karapt...|             false|\x0108d17add1f42b...|     null|6844728|    0|\x08d17add1f42b99...|          208241|2851668|  11684759477|\x29078a8215c1e11...|
|           null|Ae2tdPwUPEZ45YFwo...|             false|\x82d818582183581...|  

qr4 = "select qr3.address_output,qr_in.address from qr3 inner join qr_in on qr3.id = qr_in.id"

joined on data_hash and hash (which both represent the hash identifier of the transaction)

In [64]:
query_join_out_in = "select qr_in.address as input,qr3.address_output as output from qr_in \
    left join qr3 on qr_in.hash = qr3.hash"

In [65]:
spark.sql(query_join_out_in).createOrReplaceTempView("query_join_out_in")

In [66]:
spark.sql(query_join_out_in).show()

+--------------------+--------------------+
|               input|              output|
+--------------------+--------------------+
|Ae2tdPwUPEZ317wJQ...|Ae2tdPwUPEZHrU1Jh...|
|Ae2tdPwUPEZ317wJQ...|addr1q8jnwa7ueyhr...|
|Ae2tdPwUPEZ4ANV1B...|addr1q9vvpv8yqn84...|
|Ae2tdPwUPEZ4ANV1B...|Ae2tdPwUPEZCmH6ft...|
|Ae2tdPwUPEZ4ANV1B...|Ae2tdPwUPEZD5kyuL...|
|Ae2tdPwUPEZ4ANV1B...|addr1qx5ggmnvn6w4...|
|Ae2tdPwUPEZ4ANV1B...|Ae2tdPwUPEZKjTJTZ...|
|Ae2tdPwUPEZ4ANV1B...|addr1q8mquly603th...|
|Ae2tdPwUPEZ9iEA3q...|addr1q9vvpv8yqn84...|
|Ae2tdPwUPEZ9iEA3q...|Ae2tdPwUPEZCmH6ft...|
|Ae2tdPwUPEZ9iEA3q...|Ae2tdPwUPEZD5kyuL...|
|Ae2tdPwUPEZ9iEA3q...|addr1qx5ggmnvn6w4...|
|Ae2tdPwUPEZ9iEA3q...|Ae2tdPwUPEZKjTJTZ...|
|Ae2tdPwUPEZ9iEA3q...|addr1q8mquly603th...|
|DdzFFzCqrhsgvV4Hx...|DdzFFzCqrhspY4hae...|
|DdzFFzCqrhsgvV4Hx...|addr1qyq5exeyldtf...|
|Ae2tdPwUPEYwNguM7...|Ae2tdPwUPEYwNguM7...|
|Ae2tdPwUPEYwNguM7...|addr1qyct3t0hql3j...|
|Ae2tdPwUPEZMhScXD...|Ae2tdPwUPEZMhScXD...|
|Ae2tdPwUPEZMhScXD...|DdzFFzCqrh

In [ ]:
query_tx_test = "select tx.*,tx_out.value from tx left join tx_out on tx_out.tx_id = tx.id"

In [159]:
spark.sql(query_tx_test).show()

+---------------+--------+-----------+-------+------+--------------------+-------+--------------+-----------------+--------------+-----------+----+--------------+-----+
|_corrupt_record|block_id|block_index|deposit|   fee|                hash|     id|invalid_before|invalid_hereafter|       out_sum|script_size|size|valid_contract|value|
+---------------+--------+-----------+-------+------+--------------------+-------+--------------+-----------------+--------------+-----------+----+--------------+-----+
|           null| 4201806|          3|      0|180563|\x34f3974bfdae6dd...|2097736|          null|             null|  197640589884|          0| 262|          true| null|
|           null| 4255851|          0|      0|920000|\x178952bb71c595e...|2161769|          null|             null|  216577479905|          0| 154|          true| null|
|           null| 4255851|          1|      0|180563|\xb06c636b37622c3...|2161770|          null|             null|   19943104103|          0| 229|        

## Query 1

This just queries the UTxO set for unspent transaction outputs. It does not include staking rewards that have have not yet been withdrawn. Before being withdrawn rewards exist in ledger state and not on-chain.

In [27]:
query = "select sum (value) / 1000000 as current_supply from tx_out as tx_outer where \
    not exists \
      ( select tx_out.id from tx_out inner join tx_in \
          on tx_out.tx_id = tx_in.tx_out_id and tx_out.index = tx_in.tx_out_index \
          where tx_outer.id = tx_out.id \
      )"

In [28]:
spark.sql(query).show()

+------------------+
|    current_supply|
+------------------+
|7.73176216167149E8|
+------------------+



In [71]:
network_in_query = "select tx_out.* from tx_out \
    inner join tx_in on tx_out.tx_id = tx_in.tx_out_id \
    inner join tx on tx.id = tx_in.tx_in_id and tx_in.tx_out_index = tx_out.index"

In [72]:
spark.sql(network_in_query).createOrReplaceTempView("query_tx_in")

In [77]:
spark.sql(network_in_query).show()

+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+
|_corrupt_record|             address|address_has_script|         address_raw|data_hash|     id|index|        payment_cred|stake_address_id|  tx_id|        value|
+---------------+--------------------+------------------+--------------------+---------+-------+-----+--------------------+----------------+-------+-------------+
|           null|Ae2tdPwUPEZMhScXD...|             false|\x82d818582183581...|     null|6844727|    1|                null|            null|2851667| 636988708079|
|           null|addr1qyydz7karapt...|             false|\x0108d17add1f42b...|     null|6844728|    0|\x08d17add1f42b99...|          208241|2851668|  11684759477|
|           null|Ae2tdPwUPEZ45YFwo...|             false|\x82d818582183581...|     null|6844731|    0|                null|            null|2851671|    731400000|
|           null|DdzFF

In [103]:
network_out_query = "select tx_out.* from tx_out inner join tx on tx_out.tx_id = tx.id \
where tx_out.value = 636988708079"

In [104]:
spark.sql(network_out_query).createOrReplaceTempView("query_tx_out")

In [105]:
spark.sql(network_out_query).show()

+---------------+--------------------+------------------+--------------------+---------+-------+-----+------------+----------------+-------+------------+
|_corrupt_record|             address|address_has_script|         address_raw|data_hash|     id|index|payment_cred|stake_address_id|  tx_id|       value|
+---------------+--------------------+------------------+--------------------+---------+-------+-----+------------+----------------+-------+------------+
|           null|Ae2tdPwUPEZMhScXD...|             false|\x82d818582183581...|     null|6844727|    1|        null|            null|2851667|636988708079|
+---------------+--------------------+------------------+--------------------+---------+-------+-----+------------+----------------+-------+------------+



In [84]:
spark.sql(network_out_query).toPandas()

,_corrupt_record,address,address_has_script,address_raw,data_hash,id,index,payment_cred,stake_address_id,tx_id,value
0,None,addr1qx5dhehk7edvln55e30k66kxs9azpncwsnzhjnqcc...,False,\x01a8dbe6f6f65acfce94cc5f6d6ac6817a20cf0e84c5...,None,6844725,0,\xa8dbe6f6f65acfce94cc5f6d6ac6817a20cf0e84c579...,139423.0,2851666,865644802
1,None,DdzFFzCqrhsdwfxt4KH6dRXgTt12nmheJCHHgE165xuZbB...,False,\x82d818584283581c013ebd04a3a783ec516b83da4d08...,None,6844726,0,None,NaN,2851667,10000000
2,None,Ae2tdPwUPEZMhScXDGZ6F3dCXyq1grDphFGeEGeAGPrYtM...,False,\x82d818582183581cf9758b46c9ffb19405d8b56579a2...,None,6844727,1,None,NaN,2851667,636988708079
3,None,addr1qyydz7karaptnyq8wk8csj7vy55lc5sfn48xn5ya8...,False,\x0108d17add1f42b99007758f884bcc2529fc52099d4e...,None,6844728,0,\x08d17add1f42b99007758f884bcc2529fc52099d4e69...,208241.0,2851668,11684759477
4,None,addr1q9gjp5cftwu7zlt56qd4xgvyuygkejgafxt2t6mtd...,False,\x015120d3095bb9e17d74d01b532184e1116cc91d4996...,None,6844729,0,\x5120d3095bb9e17d74d01b532184e1116cc91d4996a5...,184323.0,2851669,474831132251
...,...,...,...,...,...,...,...,...,...,...,...
20366,None,addr1q90pvaa7ynuxmm6nay7dptvsj2wqt4cv5f3p4gchz...,False,\x015e1677be24f86def53e93cd0ad90929c05d70ca262...,None,6865091,0,\x5e1677be24f86def53e93cd0ad90929c05d70ca2621a...,134481.0,2861662,100478769
20367,None,addr1q8tg0x4g3d3yjaxfcta3slh57glvtvtue0jh2crcq...,False,\x01d6879aa88b624974c9c2fb187ef4f23ec5b17ccbe5...,None,6865092,0,\xd6879aa88b624974c9c2fb187ef4f23ec5b17ccbe575...,7534.0,2861663,25616281189
20368,None,addr1q86hd75e53dqquqycz8z7w0umdmvzwlt5mlms9uaz...,False,\x01f576fa99a45a007004c08e2f39fcdb76c13beba6ff...,None,6865093,0,\xf576fa99a45a007004c08e2f39fcdb76c13beba6ffb8...,134221.0,2861664,5784018401
20369,None,DdzFFzCqrht2yPjaMPrKhbfDbqGJ87y8NwTeUj1rmYdb73...,False,\x82d818584283581cb146444053c85bd719645b1dd629...,None,6865094,0,None,NaN,2861665,990000000


In [79]:
join_input_output = "select query_tx_out.address, query_tx_in.address, query_tx_out.value, query_tx_in.value \
from query_tx_out inner join query_tx_in on query_tx_out.tx_id = query_tx_in.tx_id"

In [80]:
spark.sql(join_input_output).show()

+--------------------+--------------------+-------------+-------------+
|             address|             address|        value|        value|
+--------------------+--------------------+-------------+-------------+
|DdzFFzCqrht5Sqtnc...|DdzFFzCqrht5Sqtnc...|1756081565310|1756081565310|
|addr1q8dv6qem5quf...|DdzFFzCqrht5Sqtnc...|    409131250|1756081565310|
|addr1q9dd0aa4d4ug...|DdzFFzCqrhspniMbz...|      1490302|    672000000|
|DdzFFzCqrhspniMbz...|DdzFFzCqrhspniMbz...|    672000000|    672000000|
|addr1vy3yv3l3h3v2...|addr1vy3yv3l3h3v2...|  16047620070|  16047620070|
|addr1q8j2xlxczt5d...|DdzFFzCqrhsk98RQk...|    209474304|  67947787692|
|DdzFFzCqrhsk98RQk...|DdzFFzCqrhsk98RQk...|  67947787692|  67947787692|
|addr1q8q0sk3dzzsj...|DdzFFzCqrhtCBBo9n...|    748860669|  79623680268|
|DdzFFzCqrhtCBBo9n...|DdzFFzCqrhtCBBo9n...|  79623680268|  79623680268|
|DdzFFzCqrhsqHhiMk...|DdzFFzCqrhsqHhiMk...|    754261825|    754261825|
|Ae2tdPwUPEZ6xYrxC...|Ae2tdPwUPEZ6xYrxC...| 119132741486| 119132

## Query 2

In [28]:
stake_address = spark.read.json(path0+"/stake_address.json")
stake_address.createOrReplaceTempView("stake_address")

In [31]:
reward = spark.read.json(path0+"/reward.json")
reward.createOrReplaceTempView("reward")

In [35]:
pool_hash = spark.read.json(path0+"/pool_hash.json")
pool_hash.createOrReplaceTempView("pool_hash")

Get the reward history for a specified stake address

In [36]:
query2 = "select reward.earned_epoch, pool_hash.view as delegated_pool, reward.amount as lovelace \
    from reward inner join stake_address on reward.addr_id = stake_address.id \
    inner join pool_hash on reward.pool_id = pool_hash.id \
    where stake_address.view = 'stake1u8gsndukzghdukmqdsd7r7wd6kvamvjv2pzcgag8v6jd69qfqyl5h' \
    order by earned_epoch asc ;"

In [37]:
spark.sql(query2).show()

+------------+--------------------+--------+
|earned_epoch|      delegated_pool|lovelace|
+------------+--------------------+--------+
|         212|pool1hwlghkwnjsjk...| 2953284|
|         213|pool1hwlghkwnjsjk...| 3333940|
|         214|pool1xxhs2zw5xa4g...| 3005843|
|         215|pool1xxhs2zw5xa4g...| 3552293|
|         216|pool1hwlghkwnjsjk...| 3130673|
|         217|pool15yyxtkhz64p7...|34339994|
|         218|pool15yyxtkhz64p7...|30384189|
|         219|pool15yyxtkhz64p7...|27293239|
|         220|pool15yyxtkhz64p7...|36947267|
|         221|pool15yyxtkhz64p7...|27016952|
|         222|pool15yyxtkhz64p7...| 7416908|
|         223|pool15yyxtkhz64p7...| 6799736|
|         224|pool15yyxtkhz64p7...|   20142|
|         225|pool15yyxtkhz64p7...|   27616|
|         226|pool15yyxtkhz64p7...|   22366|
|         227|pool15yyxtkhz64p7...|   30486|
|         228|pool15yyxtkhz64p7...|   30880|
|         229|pool15yyxtkhz64p7...|   27037|
|         230|pool15yyxtkhz64p7...|   28200|
|         

Get the block number of blocks created in an epoch by a specified pool

In [27]:
query3 = "select block.block_no, block.epoch_no, pool_hash.view as pool_view \
    from block inner join slot_leader on block.slot_leader_id = slot_leader.id \
    inner join pool_hash on slot_leader.pool_hash_id = pool_hash.id \
    where block.epoch_no = 220 \
    and pool_hash.view = 'pool137x32lrkprphrd0aa8x4jqz98z6lc0wawlc88hdjeps4qe408ad' ;"

In [28]:
spark.sql(query3).show()

AnalysisException: Table or view not found: block; line 1 pos 76;
'Project ['block.block_no, 'block.epoch_no, 'pool_hash.view AS pool_view#1333]
+- 'Filter (('block.epoch_no = 220) AND ('pool_hash.view = pool137x32lrkprphrd0aa8x4jqz98z6lc0wawlc88hdjeps4qe408ad))
   +- 'Join Inner, ('slot_leader.pool_hash_id = 'pool_hash.id)
      :- 'Join Inner, ('block.slot_leader_id = 'slot_leader.id)
      :  :- 'UnresolvedRelation [block], [], false
      :  +- 'UnresolvedRelation [slot_leader], [], false
      +- 'UnresolvedRelation [pool_hash], [], false


- problem: how to extract specific fields from the tables (only the generic ones are present)

### GRAPHFRAMES

Below, you can see a test of GraphFrames framework to create graph structures.

In [29]:
vertices = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

In [42]:
edges = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])

- problem: dependencies - solved

In [44]:
g = GraphFrame(vertices, edges)
print(g)

GraphFrame(v:[id: string, name: string ... 1 more field], e:[src: string, dst: string ... 1 more field])


In [45]:
g.vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [46]:
g.edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [47]:
vertexInDegrees = g.inDegrees

In [48]:
# Find the youngest user's age in the graph.
# This queries the vertex DataFrame.
g.vertices.groupBy().min("age").show()

+--------+
|min(age)|
+--------+
|      29|
+--------+



In [49]:
# Count the number of "follows" in the graph.
# This queries the edge DataFrame.
numFollows = g.edges.filter("relationship = 'follow'").count()

In [50]:
numFollows

4

## Motif finding: Motifs finding is also known as graph pattern matching.

In [51]:
# Search for pairs of vertices with edges in both directions between them.
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
motifs.show()

+----------------+--------------+----------------+--------------+
|               a|             e|               b|            e2|
+----------------+--------------+----------------+--------------+
|{c, Charlie, 30}|{c, b, follow}|    {b, Bob, 36}|{b, c, follow}|
|    {b, Bob, 36}|{b, c, follow}|{c, Charlie, 30}|{c, b, follow}|
+----------------+--------------+----------------+--------------+



PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

In [52]:
# Run PageRank until convergence to tolerance "tol".
results = g.pageRank(resetProbability=0.15, tol=0.01)
# Display resulting pageranks and final edge weights
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

# Run PageRank for a fixed number of iterations.
results2 = g.pageRank(resetProbability=0.15, maxIter=10)

# Run PageRank personalized for vertex "a"
results3 = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="a")

# Run PageRank personalized for vertex ["a", "b", "c", "d"] in parallel
results4 = g.parallelPersonalizedPageRank(resetProbability=0.15, sourceIds=["a", "b", "c", "d"], maxIter=10)

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  g| 0.1799821386239711|
|  f| 0.3283606792049851|
|  e|0.37085233187676075|
|  d| 0.3283606792049851|
|  c| 2.6878300011606218|
|  b|  2.655507832863289|
|  a|0.44910633706538744|
+---+-------------------+

+---+---+------+
|src|dst|weight|
+---+---+------+
|  f|  c|   1.0|
|  e|  f|   0.5|
|  e|  d|   0.5|
|  d|  a|   1.0|
|  c|  b|   1.0|
|  b|  c|   1.0|
|  a|  e|   0.5|
|  a|  b|   0.5|
+---+---+------+



In [31]:
results = g.triangleCount()
results.select("id", "count").show()

+---+-----+
| id|count|
+---+-----+
|  a|    1|
|  b|    0|
|  c|    0|
|  d|    1|
|  e|    1|
|  f|    0|
|  g|    0|
+---+-----+

